In [ ]:
from pymedphys.mosaiq import connect, execute
connection = connect('.')

In [ ]:
# iterate through structures and display list of ROI Name = ROI Types
# accept input about which to hide

In [ ]:
from pymedphys.imports import pydicom

def bds_folder_for_series(connection, dcmseries_id:int):
    return ""

def read_reference_structure_set(connection, sit_set_id:int):
    result = execute(connection,
        """
        SELECT 
            DCMSeriesID
        FROM DCMSeries
            INNER JOIN DCMStudy ON DCMStudy.StudyID = DCMSeries.DCMStudyID
            INNER JOIN Patient ON Patient.Pat_ID1 = DCMStudy.Pat_ID1
        WHERE DCMSeries.Modality = 9
        """)    
    return pydicom.dcmread('')

In [ ]:
from pathlib import Path
examples_path = Path('g:/') / 'Data_Examples_SRO'
examples_path = Path('g:/') / 'Data_Examples_MedicalImaging'
examples_dcm_files = [path for path in examples_path.glob('**/*') if path.is_file]
print(f"Found {len(examples_dcm_files)} potential dicom files")

In [ ]:
import pydicom

def read_dcm(fns):
    for fn in fns:
        try: yield pydicom.dcmread(fn)
        except Exception: continue

datasets = list(read_dcm(examples_dcm_files[20000:50000]))
print(f"Read {len(datasets)} dicom datasets")
print(f"Found modalities: {set([ds.Modality for ds in datasets if hasattr(ds, 'Modality')])}")

In [ ]:
from pprint import pprint

uid_element_modalities = {}

def extract_uids(modality, ds):
    try:
        for el in ds:
            if el.VR == 'UI': 
                key = f"{el.tag} = {el.name}"
                if not key in uid_element_modalities:
                    uid_element_modalities[key] = { modality }
                else:
                    uid_element_modalities[key].add(modality)
            if el.VR == 'SQ': 
                for item in el: extract_uids(modality, item)
    except Exception:
        pass
    
for ds in datasets:
    if hasattr(ds, 'Modality'):
        extract_uids(ds.Modality, ds)

for uid_element in uid_element_modalities:
    print(f"{uid_element}: {uid_element_modalities[uid_element]}")

In [ ]:
from pydicom.tag import Tag
pprint([(Tag('ReferencedFrameOfReferenceSequence'), 0), Tag('FrameOfReferenceUID')])

In [ ]:
pprint([(Tag('ReferencedFrameOfReferenceSequence'), 0), 
            (Tag('RTReferencedStudySequence'), 0), 
            (Tag('RTReferencedSeriesSequence'), 0), 
            (Tag('ContourImageSequence'), 0),
            Tag('ReferencedSOPInstanceUID')])

In [ ]:
from pydicom.datadict import dictionary_VR
dictionary_VR(Tag('ReferencedFrameOfReferenceSequence'))

In [ ]:
class DicomRewriter(object):
    def __init__(self):
        self._original = {}
        self._rewritten = {}

    def get_original_dataset(self, original_sopinstance_uid):
        if original_sopinstance_uid in self._original:
            return self._original[original_sopinstance_uid]
        return None

    def get_rewritten_dataset(self, original_sopinstance_uid):
        if original_sopinstance_uid in self._rewritten:
            return self._rewritten[original_sopinstance_uid]
        if original_sopinstance_uid in self._original:
            return self._original[original_sopinstance_uid]
        return None

    def rewrite(self, original_uid, path):
        pass
